In [1]:
import pandas as pd
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
# --- Load data ---
df = pd.read_csv(os.path.join("data", "activities.csv"))
df = df[df["type"] == "Run"].copy()

In [3]:
# Parse dates
df["date"] = pd.to_datetime(df["start_date"])
df = df.sort_values("date")

# Features
df["distance_km"] = df["distance"] / 1000
df["elev_gain_m"] = df["total_elevation_gain"]
df["pace_min_per_km"] = (df["moving_time"] / 60) / df["distance_km"]

# Time of day
df["hour"] = df["date"].dt.hour

# Timezone (locations)
df["timezone_code"] = pd.Categorical(df["timezone"]).codes

# Set datetime as index for rolling calculation
df = df.set_index("date")

# Rolling mileage features
df["rolling_7d_km"] = df["distance_km"].rolling("7D").sum().fillna(0)
df["rolling_30d_km"] = df["distance_km"].rolling("30D").sum().fillna(0)

# Reset index if you want 'date' as a column again
df = df.reset_index()

features = ["distance_km","elev_gain_m","rolling_7d_km","rolling_30d_km", "hour", "timezone_code"]
X = df[features]
y = df["pace_min_per_km"]

In [4]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Train model
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [5]:
import joblib

# Save the model
joblib.dump(model, "model.pkl")

# (Optional) Save the training feature list for alignment later
joblib.dump(features, "features.pkl")

['features.pkl']